<img src="./images/DLI_Header.png" style="width: 400px;">

# Assessment

Congratulations on going through today's course! Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started! 

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at [notebook 05b](05b_presidential_doggy_door.ipynb) where we implemented transfer learning.

In [1]:
from tensorflow.keras.applications import VGG16

# Load pre-trained weights from ImageNet
base_model = VGG16(
    weights='imagenet',  # or specify the path to custom weights if available
    input_shape=(224, 224, 3),
    include_top=False  # Set to False if you want to exclude the top layers
)

## Freeze Base Model

Next, we suggest freezing the base model, as done in [notebook 05b](05b_presidential_doggy_door.ipynb). This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [2]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [3]:
from tensorflow import keras

# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

# Apply VGG16 base model
x = base_model(inputs, training=False)

# Add pooling or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)  # Choose either GlobalAveragePooling2D or Flatten

# Add final dense layer for classification (adjust number of units/classes as needed)
num_classes = 6  # Example: 6 classes
outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [4]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,717,766 (56.14 MB)

 Trainable params: 3,078 (12.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [5]:
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=[keras.metrics.CategoricalAccuracy()])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). If you'd like a reference, you can check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [7]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='categorical', 
                                      batch_size=8)

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [8]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

Epoch 1/20


C:\Users\RESTU\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\backend\tensorflow\nn.py:560: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
C:\Users\RESTU\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 702ms/step - categorical_accuracy: 0.2156 - loss: 5.8476 - val_categorical_accuracy: 0.2812 - val_loss: 3.6661
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 626ms/step - categorical_accuracy: 0.3910 - loss: 2.6692 - val_categorical_accuracy: 0.4688 - val_loss: 2.6041
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 645ms/step - categorical_accuracy: 0.4383 - loss: 2.4001 - val_categorical_accuracy: 0.3438 - val_loss: 2.2259
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 659ms/step - categorical_accuracy: 0.6238 - loss: 1.5289 - val_categorical_accuracy: 0.5312 - val_loss: 2.1284
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 634ms/step - categorical_accuracy: 0.5472 - loss: 1.5596 - val_categorical_accuracy: 0.4688 - val_loss: 2.1276
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 632ms/step - categorical_accuracy: 0.6171 - loss: 1.3407 - val_categorical_accuracy: 0.7812 - val_loss: 1.0283
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 636ms/step - categorical_accuracy: 0.6779 - loss: 0.8487 

C:\Users\RESTU\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 651ms/step - categorical_accuracy: 0.8295 - loss: 0.3167 - val_categorical_accuracy: 0.7812 - val_loss: 0.7628
Epoch 13/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 294ms/step - categorical_accuracy: 0.8568 - loss: 0.4495 - val_categorical_accuracy: 0.7500 - val_loss: 0.7012
Epoch 14/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 635ms/step - categorical_accuracy: 0.8337 - loss: 0.4585 - val_categorical_accuracy: 0.7812 - val_loss: 0.9927
Epoch 15/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 654ms/step - categorical_accuracy: 0.8989 - loss: 0.2844 - val_categorical_accuracy: 0.8438 - val_loss: 0.5412
Epoch 16/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 634ms/step - categorical_accuracy: 0.9111 - loss: 0.2303 - val_categorical_accuracy: 0.7812 - val_loss: 0.6392
Epoch 17/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 645ms/step - categorical_accuracy: 0.7905 - loss: 0.4119 - val_categorical_accuracy: 0.8125 - val_loss: 0.6521
Epoch 18/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 641ms/step - categorical_accuracy: 0.7907 - loss: 0.

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [10]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [11]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - categorical_accuracy: 0.9411 - loss: 0.2596 - val_categorical_accuracy: 0.8438 - val_loss: 1.0554
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - categorical_accuracy: 0.9682 - loss: 0.1062 - val_categorical_accuracy: 0.9375 - val_loss: 0.1394
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - categorical_accuracy: 0.8778 - loss: 0.3285 - val_categorical_accuracy: 0.7812 - val_loss: 0.7330
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - categorical_accuracy: 0.9557 - loss: 0.0882 - val_categorical_accuracy: 0.9062 - val_loss: 0.2320
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - categorical_accuracy: 0.9048 - loss: 0.4098 - val_categorical_accuracy: 0.9062 - val_loss: 0.4329
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - categorical_accuracy: 0.9681 - loss: 0.0680 - val_categorical_accuracy: 0.9688 - val_loss: 0.2377
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - categorical_accuracy: 0.9461 - loss: 0.2804 - va

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [13]:
batch_size = valid_it.batch_size
steps = int(valid_it.samples / batch_size)

model.evaluate(valid_it, steps=steps)

41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 450ms/step - categorical_accuracy: 0.9256 - loss: 0.2860


[0.2075675129890442, 0.9420731663703918]

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your validation data iterator is called `valid_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [ ]:
from run_assessment import run_assessment

In [ ]:
run_assessment(model, valid_it)

## Generate a Certificate

If you passed the assessment, please return to the course page (shown below) and click the "ASSESS TASK" button, which will generate your certificate for the course.

<img src="./images/assess_task.png" style="width: 800px;">